# Data Preparation Notebook
# 
# Phase: Data cleansing
# 
### **Objective:** identify and correct errors and inconsistencies in the dataset to improve its quality and reliability.

### Setup and Imports

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
import re
import requests
from dotenv import load_dotenv



In [36]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

### Load the Data

In [ ]:
DATA_PATH = "../data/raw/all_california_gym_reviews.csv"
if not os.path.exists(DATA_PATH):
    print(f"Error: File not found at {DATA_PATH}")
else:
    # Load the data
    df = pd.read_csv(DATA_PATH, encoding='utf-8-sig')
    print(f"Successfully loaded {len(df):,} reviews")

Successfully loaded 1,165 reviews


##  1.Data cleansing 
### Handling missing values

In [97]:
print("MISSING VALUES")

# For rating column, count NaN values + 0 values as missing
# For other columns, count only NaN values
missing_counts = {}

for column in df.columns:
    if column == 'rating':
        # Count NaN + 0 values for rating
        nan_count = df[column].isna().sum()
        zero_count = (df[column] == 0).sum()
        missing_counts[column] = nan_count + zero_count
    else:
        # Count only NaN for other columns
        missing_counts[column] = df[column].isna().sum()

# Convert to Series
missing = pd.Series(missing_counts)
missing_percent = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percent': missing_percent.values
})

print(missing_df.to_string(index=False))

MISSING VALUES
  Column  Missing   Percent
      id        0  0.000000
    name        0  0.000000
  source        0  0.000000
location        0  0.000000
    date        1  0.085837
  rating       41  3.519313
 comment      605 51.931330


## Handling missing comments
To fill the 51.93% missing values in the comment field, we generate synthetic gym reviews using the Meta-Llama-3-8B-Instruct model.
Each comment is conditioned on the rating and location, with randomized style and focus parameters to ensure diversity.
A custom prompt builder creates a short, natural review (1–2 sentences), and the model output is inserted back into the dataset before exporting the completed CSV.

In [90]:

import random
import time
API_TOKEN = os.getenv("HF_TOKEN")
MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"  # modèle chat

client = InferenceClient(api_key=API_TOKEN)

# --- Définition des variations de prompts ---
styles = [
    "enthousiaste", "neutre", "humoristique", "concise", "sincère",
    "positif", "constructif", "critique légère", "motivant", "amical", ""
]
focus = [
    "l'ambiance", "les équipements", "les coachs", "la propreté", "la diversité des cours",
    "l'espace et confort", "la musique et l'atmosphère", "le prix et les abonnements", 
    "la disponibilité des machines", "les horaires et flexibilité", "le suivi personnalisé", ""
]


def random_prompt(stars, location):
    style = random.choice(styles)
    f = random.choice(focus)

    prompt = f"Génère un commentaire court pour une salle de sport notée {stars}/5. Lieu : {location}."
    
    if style.strip():
        prompt += f" Style : {style}."
    if f.strip():
        prompt += f" Mets l'accent sur {f}."
    
    prompt += " Le commentaire doit être naturel et convaincant, 1 à 2 phrases maximum."
    
    return prompt


# --- Fonction pour générer un commentaire ---
def generate_comment_chat(stars, location):
    prompt = random_prompt(stars, location)
    messages = [{"role": "user", "content": prompt}]
    
    try:
        response = client.chat_completion(model=MODEL, messages=messages, max_tokens=80)  # max_tokens réduit pour commentaire court
        return response.choices[0].message["content"]
    except Exception as e:
        print("Erreur génération :", e)
        return None

# --- Remplir les commentaires manquants ---
for idx, row in df.iterrows():
    stars = row['rating']
    comment_missing = pd.isna(row['comment']) or row['comment'].strip() == ""
    location = row['location']
    
    if comment_missing and stars is not None and stars != 0 and location is not None:
        comment = generate_comment_chat(stars, location)
        if comment:
            # garder uniquement la première phrase si besoin
            comment = comment.split('.')[0].strip() + '.'
            df.at[idx, 'comment'] = comment
            print(f"Commentaire généré pour l'index {idx} avec {stars} étoiles : {comment}")
        else:
            print(f"Échec de la génération pour l'index {idx}.")
        time.sleep(1)  # pause pour limiter le rythme des appels API

# --- Sauvegarder le dataset complété ---
df.to_csv("C:\\Users\\MSI\\Desktop\\projet_ml\\customer_review_analysis\\data\\raw\\reviews_all_filled.csv", index=False)
print("Dataset complété et sauvegardé !")


Dataset complété et sauvegardé !


In [91]:
print("MISSING VALUES")

# For rating column, count NaN values + 0 values as missing
# For other columns, count only NaN values
missing_counts = {}

for column in df.columns:
    if column == 'rating':
        # Count NaN + 0 values for rating
        nan_count = df[column].isna().sum()
        zero_count = (df[column] == 0).sum()
        missing_counts[column] = nan_count + zero_count
    else:
        # Count only NaN for other columns
        missing_counts[column] = df[column].isna().sum()
# Convert to Series
missing = pd.Series(missing_counts)
missing_percent = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percent': missing_percent.values
})

print(missing_df.to_string(index=False))

MISSING VALUES
  Column  Missing  Percent
      id        0 0.000000
    name        0 0.000000
  source        0 0.000000
location        0 0.000000
    date        1 0.085837
  rating       41 3.519313
 comment        1 0.085837


### Handling Missing Values for Ratings

Although the comment field contains the highest proportion of missing entries, the rating column also includes missing or zero values. To ensure consistency in our sentiment-based analyses, we generate synthetic ratings from the textual content of the comments.

We use an LLM-based function that infers a rating (1–5) directly from the sentiment expressed in each comment.

In [94]:
# --- Fonction pour générer une note à partir d'un commentaire ---
def generate_rating_from_comment(comment):
    prompt = (
        f"En te basant sur ce commentaire, donne une note entière entre 1 et 5 étoiles pour la salle de sport :\n"
        f"{comment}\nRéponds uniquement par le chiffre."
    )
    messages = [{"role": "user", "content": prompt}]
    
    try:
        response = client.chat_completion(model=MODEL, messages=messages, max_tokens=10)
        rating_text = response.choices[0].message["content"].strip()
        # essayer de convertir en int
        rating = int(rating_text[0])  # prend le premier chiffre trouvé
        return rating
    except Exception as e:
        print("Erreur génération rating :", e)
        return None

# --- Remplir les ratings manquants ---
for idx, row in df.iterrows():
    rating_missing = row['rating'] is None or row['rating'] == 0
    comment = row['comment']
    
    if rating_missing and comment :
        rating = generate_rating_from_comment(comment)
        if rating:
            df.at[idx, 'rating'] = rating
            print(f"Rating généré pour l'index {idx} : {rating} étoiles , comment: {comment}")
        else:
            print(f"Échec de la génération du rating pour l'index {idx}")
        time.sleep(1)  # limiter le rythme des appels API

# --- Sauvegarder le dataset complété ---
df.to_csv("C:\\Users\\MSI\\Desktop\\projet_ml\\customer_review_analysis\\data\\raw\\reviews_all_filled.csv", index=False)
print("Dataset complété et sauvegardé !")


Rating généré pour l'index 1115 : 1 étoiles , comment: nan
Rating généré pour l'index 1141 : 5 étoiles , comment: Ok pourquoi pas! On se capte en privé. A+ Soumy
Rating généré pour l'index 1142 : 4 étoiles , comment: Perso, je vais chez Ladies Gym à la Marsa (au-dessus de Topnet). C'est surtout axé sur un circuit de 30 minutes en alternant machines et exercices avec un coach. C'est réservé aux femmes. J'aime bien, c'est une petite salle tranquille
Rating généré pour l'index 1143 : 3 étoiles , comment: Mamourette, tu peux me dire de combien est l'abonnement?
Rating généré pour l'index 1144 : 4 étoiles , comment: Pour l'instant, j'ai pris un mois (c'est moins cher si on prend pour 6 mois: circuit de 30 minutes à toute heure + cours (pilates et plein d'autres trucs): 110 DT / mois. Ladies Gym a une page FB. Je crois que les tarifs sont dessus.
Rating généré pour l'index 1145 : 3 étoiles , comment: Vous avez une autre salle réservée aux femmes au niveau du lac 2 si c'est plus proche pour v

In [120]:
print("MISSING VALUES")

# For rating column, count NaN values + 0 values as missing
# For other columns, count only NaN values
missing_counts = {}

for column in df.columns:
    if column == 'rating':
        # Count NaN + 0 values for rating
        nan_count = df[column].isna().sum()
        zero_count = (df[column] == 0).sum()
        missing_counts[column] = nan_count + zero_count
    else:
        # Count only NaN for other columns
        missing_counts[column] = df[column].isna().sum()
# Convert to Series
missing = pd.Series(missing_counts)
missing_percent = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percent': missing_percent.values
})

print(missing_df.to_string(index=False))

MISSING VALUES
  Column  Missing  Percent
      id        0 0.000000
    name        0 0.000000
  source        0 0.000000
location        0 0.000000
    date        1 0.085837
  rating        1 0.085837
 comment        1 0.085837


The dataset contains very few missing entries, with the comment and rating fields each showing 0.08% missing values.
### Rating Normalisation : 
we normalize the rating field by enforcing valid values between 1 and 5 stars.

In [121]:
abnormal_mask = (df['rating'] < 1) | (df['rating'] > 5) | (df['rating'].isna())

print("number of abnormal ratings before normalization:", abnormal_mask.sum())

df['rating'] = df['rating'].clip(lower=1, upper=5).fillna(3)
abnormal_mask = (df['rating'] < 1) | (df['rating'] > 5) | (df['rating'].isna())
print("number of abnormal ratings after normalization:", abnormal_mask.sum())


number of abnormal ratings before normalization: 2
number of abnormal ratings after normalization: 0


In [123]:
df.to_csv("C:\\Users\\MSI\\Desktop\\projet_ml\\customer_review_analysis\\data\\raw\\reviews_all_filled.csv", index=False)
